In [1]:
# 1. Pre-processing of the dataset
# 2. Use all Diagnostics techniques explained in the lecture
# 3. Compare between the performance of ALL the following classifiers: KNN / Naive Bayes / Logistic Regression / Neural Networks

In [2]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from Classification_Project_DE_Module import DE_fn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score

In [3]:
if os.path.exists('./2018_Accidents_UK_Final.csv'):
    x = np.nan
else:
    DE_fn()

In [4]:
df = pd.read_csv('./2018_Accidents_UK_Final.csv')

C:\Users\ahmad\AppData\Local\Temp\ipykernel_1476\2076121321.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./2018_Accidents_UK_Final.csv')


Index(['accident_index', 'accident_year', 'accident_reference',
       'location_easting_osgr', 'location_northing_osgr', 'longitude',
       'latitude', 'accident_severity', 'number_of_vehicles',
       'number_of_casualties', 'date', 'time', 'first_road_number',
       'speed_limit', 'second_road_class', 'second_road_number',
       'lsoa_of_accident_location', 'dateTime', 'week_end', 'time_in_mins',
       'police_force_0', 'police_force_1', 'police_force_2', 'police_force_3',
       'police_force_4', 'police_force_5', 'day_of_week_0', 'day_of_week_1',
       'day_of_week_2', 'local_authority_district_0',
       'local_authority_district_1', 'local_authority_district_2',
       'local_authority_district_3', 'local_authority_district_4',
       'local_authority_district_5', 'local_authority_district_6',
       'local_authority_district_7', 'local_authority_district_8',
       'local_authority_ons_district_0', 'local_authority_ons_district_1',
       'local_authority_ons_district_2', 

In [5]:
df_copy = df[['number_of_vehicles','number_of_casualties','day_of_week_0','day_of_week_1','day_of_week_2',
        'longitude','latitude','first_road_class_0','first_road_class_1','first_road_class_2','road_type_0','road_type_1',
        'road_type_2','junction_detail_0','junction_detail_1','junction_detail_2','junction_detail_3',
        'junction_control_0','junction_control_1','junction_control_2','pedestrian_crossing_human_control_0',
        'pedestrian_crossing_human_control_1','pedestrian_crossing_physical_facilities_0',
        'pedestrian_crossing_physical_facilities_1','pedestrian_crossing_physical_facilities_2','light_conditions_0',
        'light_conditions_1','light_conditions_2','weather_conditions_0','weather_conditions_1','weather_conditions_2',
        'road_surface_conditions_0','road_surface_conditions_1','trunk_road_flag_0','trunk_road_flag_1',
        'week_number_0', 'week_number_1', 'week_number_2','week_number_3', 'week_number_4',
        'week_number_5','special_conditions_at_site_None','special_conditions_at_site_Rare',
        'carriageway_hazards_None', 'carriageway_hazards_Rare','urban_or_rural_area_Rural',
        'urban_or_rural_area_Urban','week_end', 'time_in_mins','time_discrete_0','time_discrete_1', 'time_discrete_2']]

In [6]:
# We will write here the feature that will be feeded into the models
X = df_copy
y = df['accident_severity']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

k = round(math.sqrt(len(df_copy)))
knn = KNeighborsClassifier(n_neighbors=k)

cv_scores = cross_val_score(knn, X_train, y_train, cv=5)
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Cross-Validation Scores: [0.79940274 0.79940274 0.7994439  0.79939241 0.79939241]
Mean CV Accuracy: 0.7994068399621201
Accuracy: 0.7976685751946286


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)
y_pred = naive_bayes.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.4770770688305804


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7976685751946286


C:\Users\ahmad\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

input_dim = X_train.shape[1]
num_classes = len(set(y_train))

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30, batch_size=1000, verbose=1)

_, accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy:", accuracy)

Epoch 1/30
98/98 [==============================] - 1s 3ms/step - loss: 2.7808 - accuracy: 0.7083
Epoch 2/30
98/98 [==============================] - 0s 3ms/step - loss: 0.9665 - accuracy: 0.7012
Epoch 3/30
98/98 [==============================] - 0s 3ms/step - loss: 1.0495 - accuracy: 0.6985
Epoch 4/30
98/98 [==============================] - 0s 3ms/step - loss: 0.7847 - accuracy: 0.7426
Epoch 5/30
98/98 [==============================] - 0s 3ms/step - loss: 0.8189 - accuracy: 0.7533
Epoch 6/30
98/98 [==============================] - 0s 3ms/step - loss: 1.0283 - accuracy: 0.6997
Epoch 7/30
98/98 [==============================] - 0s 3ms/step - loss: 0.6281 - accuracy: 0.7783
Epoch 8/30
98/98 [==============================] - 0s 3ms/step - loss: 0.6281 - accuracy: 0.7866
Epoch 9/30
98/98 [==============================] - 0s 3ms/step - loss: 0.8303 - accuracy: 0.7459
Epoch 10/30
98/98 [==============================] - 0s 3ms/step - loss: 0.5746 - accuracy: 0.7918
Epoch 11/30
98/98 [